Note to users:
This notebook / repo was intended to be provided to users to check the analysis of the Hockenberry et al. (2017) Open Biology paper. In its current form it serves that purpose.

That being said, it's not exactly clean or really intended as a platform for extensions and analysis of novel datasets. More accurately, this code / these notebooks became a leviathan of just trying to cram everything into the right sized figures to carry the paper across the finish line. This paper was a discovery paper and not a tool, so all of the functions are in here and usable but they're buried within these libraries and not exactly clean. Feel free to email me if you'd like to actually analyze new datasets and I can provide you with a 1000x nicer notebook/set of functions to perform a basic analysis on your dataset of interest. Unfortunately the figures for an actual paper always get messier / more complex so much in this notebook might not be entirely useful for individuals. 

Also as of the last update to these notebooks, where I've ensured that everything runs properly and free of errors, I've noticed that this notebook makes pretty poor figures that need some adjustments to legends, axes, etc owing to updates undoubtedly in statsmodels, matplotlib, etc. Apologies but I leave this as an exercise to the user to debug if they want cleanliness. This notebook was created for openness and reproducibility only.

**I stress, however, that if you want to use this type of analysis in any future work please reach out to me and I'd be ecstatic to collaborate and/or provide you with more up-to-date, readable, and user-friendly code that will be geared towards your specific dataset and present a cleaner analysis for those purposes. That code is not written but my familiarity and interest in this project is such that it could probably be done in a few hours or days at most.**


--Adam J Hockenberry

# Some ipython magic to make life a bit easier

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

# Basic imports, more importantly please be sure to look in "Shine_Dalgarno_library" to make sure you have the required imports such that the following cell does not fail

In [ ]:
import Shine_Dalgarno_library
import json
import glob
import pandas as pd
import numpy as np

# These are the basic files depending on which organism/dataset you want to analyze

In [ ]:
#For all datasets, use the same aSD sequences in the 5' direction
asd_extensions_5_prime = ['-UGGAUCACCUCC.txt', '-GGAUCACCUCC.txt', '-GAUCACCUCC.txt',\
           '-AUCACCUCC.txt','-UCACCUCC.txt','-CACCUCC.txt','-ACCUCC.txt', '-CCUCC.txt']

Next, you can comment / uncomment your datasets at will according to which data you'd like to analyze. These aSD extensions were tuned to the organism so if you do not select a valid organism-specific aSD sequence at any point you'll probably get errors. Fragment binding energies for shorter aSD seqs will work for all possible x-mers but for the longer aSD seqs I only computed those necessary to evaluate the specific organisms of interest. I.e. AUCACCUCCUUUCU will not work for E. coli because the E. coli tail is CCUCCUUA...

In [ ]:
#For C. crescentus
# teff_file = '../Data/Ccrescentus_rich_teff_cov=75_len=90_ignore=0.json'
# mfe_file = '../Data/cis_structure_files/NA1000_JMS_MFE.json'
# genome_file = '../Data/genomes/NA1000_JMS.gbk'
# asd_extensions_3_prime = ['-CCUCC.txt', '-CCUCCU.txt', '-CCUCCUU.txt', '-CCUCCUUU.txt',\
#            '-CCUCCUUUC.txt', '-CCUCCUUUCU.txt', '-CCUCCUUUCUA.txt', '-CCUCCUUUCUAA.txt', '-CCUCCUUUCUAAG.txt']
# asd_extensions_varied = ['-GAUCACCUCCUUU.txt', '-AUCACCUCCUUU.txt', '-UCACCUCCUUU.txt', '-CACCUCCUUU.txt', '-ACCUCCUUU.txt', '-CCUCCUUU.txt',\
#            '-GAUCACCUCCUUUC.txt', '-AUCACCUCCUUUC.txt', '-UCACCUCCUUUC.txt', '-CACCUCCUUUC.txt', '-ACCUCCUUUC.txt', '-CCUCCUUUC.txt',\
#            '-GAUCACCUCCUUUCU.txt', '-AUCACCUCCUUUCU.txt', '-UCACCUCCUUUCU.txt', '-CACCUCCUUUCU.txt', '-ACCUCCUUUCU.txt', '-CCUCCUUUCU.txt',\
#            '-GAUCACCUCCUUUCUA.txt', '-AUCACCUCCUUUCUA.txt', '-UCACCUCCUUUCUA.txt', '-CACCUCCUUUCUA.txt', '-ACCUCCUUUCUA.txt', '-CCUCCUUUCUA.txt']

#For E. coli datasets
# teff_file = '../Data/Ecoli_rich_teff_cov=75_len=90_ignore=0.json'
teff_file = '../Data/Buskirk1_teff.json'
# teff_file = '../Data/Buskirk2_teff.json'
mfe_file = '../Data/cis_structure_files/oldEcoli_MFE.json'
genome_file = '../Data/genomes/oldEcoli.gb'
asd_extensions_3_prime = ['-CCUCC.txt', '-CCUCCU.txt', '-CCUCCUU.txt', '-CCUCCUUA.txt', '-CCUCCUUAC.txt']#For E. coli
asd_extensions_varied = ['-GAUCACCUCCUUA.txt', '-AUCACCUCCUUA.txt', '-UCACCUCCUUA.txt', '-CACCUCCUUA.txt', '-ACCUCCUUA.txt','-CCUCCUUA.txt',\
           '-GAUCACCUCCUUAC.txt', '-AUCACCUCCUUAC.txt',  '-UCACCUCCUUAC.txt', '-CACCUCCUUAC.txt', '-ACCUCCUUAC.txt','-CCUCCUUAC.txt']

#For B. subtilis
# teff_file = '../Data/Bsubtilis_rich_teff_cov=75_len=90_ignore=0.json'
# mfe_file = '../Data/cis_structure_files/bacillus_MFE.json'
# genome_file = '../Data/genomes/bacillus.gb'
# asd_extensions_3_prime = ['-CCUCC.txt', '-CCUCCU.txt', '-CCUCCUU.txt', '-CCUCCUUU.txt',\
#            '-CCUCCUUUC.txt', '-CCUCCUUUCU.txt', '-CCUCCUUUCUA.txt', '-CCUCCUUUCUAA.txt', '-CCUCCUUUCUAAG.txt']#For B. subtilis
# asd_extensions_varied = ['-GAUCACCUCCUUU.txt', '-AUCACCUCCUUU.txt', '-UCACCUCCUUU.txt', '-CACCUCCUUU.txt', '-ACCUCCUUU.txt', '-CCUCCUUU.txt',\
#            '-GAUCACCUCCUUUC.txt', '-AUCACCUCCUUUC.txt', '-UCACCUCCUUUC.txt', '-CACCUCCUUUC.txt', '-ACCUCCUUUC.txt', '-CCUCCUUUC.txt',\
#            '-GAUCACCUCCUUUCU.txt', '-AUCACCUCCUUUCU.txt', '-UCACCUCCUUUCU.txt', '-CACCUCCUUUCU.txt', '-ACCUCCUUUCU.txt', '-CCUCCUUUCU.txt',\
#            '-GAUCACCUCCUUUCUA.txt', '-AUCACCUCCUUUCUA.txt', '-UCACCUCCUUUCUA.txt', '-CACCUCCUUUCUA.txt', '-ACCUCCUUUCUA.txt', '-CCUCCUUUCUA.txt']

# From the files above, get the basic dictionaries of gene sequence, 5' UTR, translation efficiency, folding energy, etc.

If you're curious how to calculate folding energy files that I called mfe_file above, these functions are included in Shine_Dalgarno_library.py

Specifically, refer to the get_structure() function. However note that being able to run this function, or rather RNA_fold_wrapper, might require some optimization on your end to install ViennaRNA and make sure that it's listed in your bash path. Thus, I've simply provided these files for your convenience.

In [ ]:
seq_dict, five_prime_seq_dict, three_prime_seq_dict = Shine_Dalgarno_library.get_seq_dicts(genome_file)

with open(teff_file) as infile:
    trans_eff_dict = json.load(infile)
print("Number of genes with teff data:", len(trans_eff_dict.keys()))

with open(mfe_file) as infile:
    mfe_dict = json.load(infile)
print("Number of genes with mfe data:", len(mfe_dict.keys()))


to_delete = []
for gene in trans_eff_dict:
    if gene not in seq_dict:
        to_delete.append(gene)
for gene in to_delete:
    del trans_eff_dict[gene]
print("Number of genes with teff data (after some processing):", len(trans_eff_dict.keys()))


trans_eff_dict_log = {}
for i in trans_eff_dict.keys():
    trans_eff_dict_log[i] = trans_eff_dict[i]

# Correct the translation efficiency measurements by regressing against folding energy

In [ ]:
####If you want to correct for the effect of cis-structure run this code
trans_eff_dict_structure = Shine_Dalgarno_library.teff_given_structure(trans_eff_dict, mfe_dict)

####Otherwise, and I'll admit this is kind of sloppy but meh... just run the below line to pretend that
####you corrected for structure even though you didn't. Both analyses showed up in the manuscript/SI
# trans_eff_dict_sturcture = trans_eff_dict

# For a given aSD seq, calculate 1st and 3rd order polynomial fits for a variety of spacings relative to the start codon

These json-energyRef-XXXX files are all pre-computed hybridization energies between the sequence referenced in the file name XXXX and all possible equally sized partners (for the longest aSD seqs I didn't actually compute all so these files contain only the computations that were necessary given the genome of that species)

In [ ]:
energy_base = '../Data/energy_dictionaries/json-energyRef-'
asd_of_interest = 'ACCUCCUUA' 
ending = '.txt'
energy_file = energy_base + asd_of_interest + ending
print(energy_file)

spacing_list = list(range(1, 15))
Shine_Dalgarno_library.compare_first_and_third_degree(trans_eff_dict_structure,\
                                                      five_prime_seq_dict,\
                                                      seq_dict,\
                                                      energy_file,\
                                                      spacing_list)


# Plot 1st and 3rd order fits for a single spacing

In [ ]:
energy_base = '../Data/energy_dictionaries/json-energyRef-'
asd_of_interest = 'ACCUCCUUA'

ending = '.txt'
energy_file = energy_base + asd_of_interest + ending

spacing = -3

poly_1_model, poly_3_model, percents, df = Shine_Dalgarno_library.plot_individual(trans_eff_dict_structure,\
                                                        five_prime_seq_dict, seq_dict, energy_file, spacing)

# Compare the various 3' aSD extensions in the form of a heat map
### This and following are only showing 3rd order polynomial fits

In [ ]:
spacing_list = list(range(1, 15))
energy_file_list = []
for i in asd_extensions_3_prime:
    for energy_file in glob.glob('../Data/energy_dictionaries/json-energyRef*.txt'):
        if i in energy_file:
            energy_file_list.append(energy_file)

Shine_Dalgarno_library.compare_different_asds(trans_eff_dict_log, five_prime_seq_dict, seq_dict, energy_file_list,\
                                              spacing_list)

# Compare the various 5' aSD extensions in the form of a heat map

In [ ]:
spacing_list = list(range(1, 15))
energy_file_list = []
for i in asd_extensions_5_prime:
    for energy_file in glob.glob('../Data/energy_dictionaries/json-energyRef*.txt'):
        if i in energy_file:
            energy_file_list.append(energy_file)

Shine_Dalgarno_library.compare_different_asds(trans_eff_dict_structure, five_prime_seq_dict, seq_dict, energy_file_list,\
                                              spacing_list)

# Compare some combinations of 5' and 3' extensions in the form of a heat map

In [ ]:
spacing_list = list(range(1, 15))
energy_file_list = []
for i in asd_extensions_varied:
    for energy_file in glob.glob('../Data/energy_dictionaries/json-energyRef*.txt'):
        if i in energy_file:
            energy_file_list.append(energy_file)

Shine_Dalgarno_library.compare_different_asds(trans_eff_dict_structure, five_prime_seq_dict, seq_dict, energy_file_list,\
                                              spacing_list)

# That was the basic analysis / work flow that was the main point of the paper. Below I'll step through a few other figures that might be useful including analyses of non-ribosomal profiling datasets

## Analysis of the Taniguchi et al data

In [ ]:
###These translation efficiency dictionaries were derived from Taniguchi et al. with the files demarcated
###as _xxx meaning the x% lowest signal to noise datset. I.e. teff_20.json is the translation efficiencies
###of the 20% of genes from this dataset with the highest confidence translation efficiency measurements
###not necessarily the highest
teff_file_list = ['teff_all.json', 'teff_80.json', 'teff_60.json', 'teff_40.json', 'teff_20.json']
teff_file_list = ['../Data/taniguchi_data/'+i for i in teff_file_list]

genome_file = '../Data/genomes/oldEcoli.gb'
organism_name = 'Escherichia_taniguchi'

seq_dict, five_prime_seq_dict, three_prime_seq_dict = Shine_Dalgarno_library.get_seq_dicts(genome_file)

first_order_r2 = []
third_order_r2 = []
percent_increase_list = []

for teff_file in teff_file_list:
    organism_name = 'Escherichia_taniguchi'+teff_file.split('/')[-1].strip('.json').strip('teff')

    with open(teff_file) as infile:
        trans_eff_dict = json.load(infile)
    print(len(trans_eff_dict.keys()))

    to_delete = []
    for gene in trans_eff_dict:
        if gene not in seq_dict:
            to_delete.append(gene)
    for gene in to_delete:
        del trans_eff_dict[gene]
    
    with open('../Data/cis_structure_files/oldEcoli_MFE.json') as infile:
        mfe_dict = json.load(infile) 
    
    trans_eff_dict_structure = Shine_Dalgarno_library.teff_given_structure(trans_eff_dict, mfe_dict)    
    
    
    energy_base = '../Data/energy_dictionaries/json-energyRef-'
    asd_of_interest = 'ACCUCCUUA'
    ending = '.txt'
    energy_file = energy_base + asd_of_interest + ending


    flexibility = 0
    spacing = -5

    temp_1, temp_3, percent_increase, df = Shine_Dalgarno_library.plot_individual(trans_eff_dict_structure, five_prime_seq_dict, seq_dict, energy_file, spacing)
    first_order_r2.append(temp_1.rsquared_adj)
    third_order_r2.append(temp_3.rsquared_adj)
    percent_increase_list.append(percent_increase)

In [ ]:
####All this information is vomited out above if you can find it but here it is if you're interested
print(first_order_r2)
print(third_order_r2)
print(percent_increase_list)

In [ ]:
x_labels = ['', 'All data\n (n=1014)', 'Top 80\%\n(n=811)', 'Top 60\%\n(n=611)', 'Top 40\%\n(n=407)', 'Top 20\%\n(n=204)']
Shine_Dalgarno_library.plot_taniguchi_data(first_order_r2, third_order_r2, percent_increase_list, x_labels)

# Kosuri data plot

## Analysis of the Kosuri et al data

In [ ]:
df = pd.read_excel('../Data/kosuri_data/kosuri_data.xls')

In [ ]:
seq_dicty = {}
exp_dicty = {}
for i in range(111):
    seq_dicty[df.loc[i]['RBS'].strip('\"')] = str(df.loc[i]['Sequence']).strip('\"').replace(' ', '')[:-3].replace('T', 'U')
    exp_dicty[df.loc[i]['RBS'].strip('\"')] = np.log2(df.loc[i]['mean.xlat'])

try:
    del seq_dicty['DeadRBS']
except KeyError:
    pass


In [ ]:
energy_file = '../Data/energy_dictionaries/json-energyRef-ACCUCCUUA.txt'
with open(energy_file) as infile:
    energy_dict = json.load(infile)
    
asd_seq = energy_file.split('-')[-1].strip('.txt')
print(asd_seq)
spacing = -5
gene_binding_dict = {}

for i in seq_dicty.keys():
    gene_binding_dict[i] = energy_dict[seq_dicty[i][spacing-len(asd_seq):spacing]]
#     gene_binding_dict[i] = min([energy_dict[seq_dicty[i][-14:-5]], energy_dict[seq_dicty[i][-15:-6]], energy_dict[seq_dicty[i][-13:-4]]])


with open('../Data/kosuri_teffs.json', 'w') as outfile:
    json.dump(exp_dicty, outfile)
with open('../Data/kosuri_binding_energies.json', 'w') as outfile:
    json.dump(gene_binding_dict, outfile)

In [ ]:
Shine_Dalgarno_library.plot_individual_kosuri(exp_dicty, gene_binding_dict, asd_seq, spacing)

# Operon analysis
This is hacky / kind of incomplete but it basically just will give you back a "trans_eff_dict" limited by the operon positions that you want and you can then run this trans_eff_dict through whichever analysis listed above including correcting for structure, etc and perform an analysis split according to operon.

In [ ]:
with open(teff_file) as infile:
    trans_eff_dict = json.load(infile)
print("Number of genes with teff data:", len(trans_eff_dict.keys()))

to_delete = []
for gene in trans_eff_dict:
    if gene not in seq_dict:
        to_delete.append(gene)
for gene in to_delete:
    del trans_eff_dict[gene]
print("Number of genes with teff data (after some processing):", len(trans_eff_dict.keys()))




# df = pd.read_csv('../Data/operons/ccrescentus_DOOR_operons.csv', sep='\t', index_col='id')
# df = pd.read_csv('../Data/operons/bsub_DOOR_operons.csv', sep='\t', index_col='id')
df = pd.read_csv('../Data/operons/ecoli_DOOR_operons.csv', sep='\t', index_col='id')
operonDict = {}
for index in df.index:
    for number, synonym in enumerate(df.loc[index]['synonyms'].split(';')):
        if synonym.strip() in operonDict.keys():
            operonDict[synonym.strip()] = 'ambiguous'
        elif number == 0:
            operonDict[synonym.strip()] = 'First'
        else:
            operonDict[synonym.strip()] = 'Late'
            
print(len(operonDict.keys()))
print(set(operonDict.values()))            

for gene in list(trans_eff_dict.keys()):
    if gene not in operonDict.keys():
        del trans_eff_dict[gene]
    elif operonDict[gene] != 'First': #####Choose/toggle operon positions here
#     elif operonDict[gene] != 'Late': ######And here
        del trans_eff_dict[gene]
print(len(trans_eff_dict.keys()))

# organism_name = organism_name + 'first_in_TU'
organism_name = organism_name + 'middle_of_TU'